# W_LEDGER_D ETL Process
### Staging to Dimension - SCD Type 1 with User Lookups and MERGE

In [ ]:
%sql
-- 1) PARAMETERS: Derive ETL control parameters
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT
  CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE, '1' AS TENANT_ID,
  2624982 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_LEDGERDIMENSION'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING: Read from staging table W_LEDGER_DS and perform user lookups
CREATE OR REPLACE TEMP VIEW stg_ledger_with_users AS
SELECT 
  ds.name AS ledger_name, ds.short_name AS ledger_short_name, ds.description AS ledger_desc,
  COALESCE(ds.chart_of_accounts, '__NOT_APPLICABLE__') AS chart_of_accounts,
  COALESCE(ds.currency_code, '__NOT_APPLICABLE__') AS currency_code,
  ds.period_set_name AS calender_name,
  COALESCE(ds.sla_accounting_method_code, '__NOT_APPLICABLE__') AS sla_accounting_method_code,
  ds.user_period_type AS period_type,
  COALESCE(ds.ledger_category_code, '__NOT_APPLICABLE__') AS ledger_category_code,
  ds.integration_id AS set_id,
  COALESCE(u_created.row_wid, 0) AS created_by_wid,
  COALESCE(u_changed.row_wid, 0) AS changed_by_wid,
  ds.creation_date AS created_on_dt, ds.last_update_date AS changed_on_dt,
  ds.last_update_date AS aux1_changed_on_dt,
  CAST(NULL AS TIMESTAMP) AS aux2_changed_on_dt,
  CAST(NULL AS TIMESTAMP) AS aux3_changed_on_dt,
  CAST(NULL AS TIMESTAMP) AS aux4_changed_on_dt,
  CASE WHEN ds.delete_flg = 'Y' THEN 'Y' ELSE 'N' END AS delete_flg,
  ds.datasource_num_id, ds.integration_id, ds.tenant_id, '0' AS x_custom
FROM workspace.oracle_edw.w_ledger_ds ds
LEFT JOIN workspace.oracle_edw.w_user_d u_created
  ON ds.datasource_num_id = u_created.datasource_num_id
  AND ds.created_by_id = u_created.integration_id
  AND ds.creation_date >= u_created.effective_from_dt
  AND ds.creation_date < u_created.effective_to_dt
  AND u_created.delete_flg = 'N'
LEFT JOIN workspace.oracle_edw.w_user_d u_changed
  ON ds.datasource_num_id = u_changed.datasource_num_id
  AND ds.changed_by_id = u_changed.integration_id
  AND ds.last_update_date >= u_changed.effective_from_dt
  AND ds.last_update_date < u_changed.effective_to_dt
  AND u_changed.delete_flg = 'N';

In [ ]:
%sql
-- 3) CHANGE DETECTION: Insert/update staging table with change indicator
CREATE OR REPLACE TEMP VIEW stg_with_change_ind AS
SELECT s.*,
  CASE
    WHEN t.row_wid IS NOT NULL
      AND (t.changed_on_dt = s.changed_on_dt OR (t.changed_on_dt IS NULL AND s.changed_on_dt IS NULL))
      AND (t.aux1_changed_on_dt = s.aux1_changed_on_dt OR (t.aux1_changed_on_dt IS NULL AND s.aux1_changed_on_dt IS NULL))
      AND (t.aux2_changed_on_dt = s.aux2_changed_on_dt OR (t.aux2_changed_on_dt IS NULL AND s.aux2_changed_on_dt IS NULL))
      AND (t.aux3_changed_on_dt = s.aux3_changed_on_dt OR (t.aux3_changed_on_dt IS NULL AND s.aux3_changed_on_dt IS NULL))
      AND (t.aux4_changed_on_dt = s.aux4_changed_on_dt OR (t.aux4_changed_on_dt IS NULL AND s.aux4_changed_on_dt IS NULL))
    THEN 'N'  -- No change
    WHEN t.row_wid IS NOT NULL THEN 'U'  -- Update
    ELSE 'I'  -- Insert
  END AS ind_update
FROM stg_ledger_with_users s
LEFT JOIN workspace.oracle_edw.w_ledger_d t
  ON s.datasource_num_id = t.datasource_num_id
  AND s.integration_id = t.integration_id;

In [ ]:
%sql
-- 4) MERGE: Update existing and insert new records
MERGE INTO workspace.oracle_edw.w_ledger_d AS tgt
USING (
  SELECT * FROM stg_with_change_ind WHERE ind_update IN ('I', 'U')
) AS src
ON tgt.datasource_num_id = src.datasource_num_id
  AND tgt.integration_id = src.integration_id
WHEN MATCHED AND src.ind_update = 'U' THEN UPDATE SET
  tgt.ledger_name = src.ledger_name,
  tgt.ledger_short_name = src.ledger_short_name,
  tgt.ledger_desc = src.ledger_desc,
  tgt.chart_of_accounts = src.chart_of_accounts,
  tgt.currency_code = src.currency_code,
  tgt.calender_name = src.calender_name,
  tgt.sla_accounting_method_code = src.sla_accounting_method_code,
  tgt.period_type = src.period_type,
  tgt.ledger_category_code = src.ledger_category_code,
  tgt.set_id = src.set_id,
  tgt.created_by_wid = src.created_by_wid,
  tgt.changed_by_wid = src.changed_by_wid,
  tgt.created_on_dt = src.created_on_dt,
  tgt.changed_on_dt = src.changed_on_dt,
  tgt.aux1_changed_on_dt = src.aux1_changed_on_dt,
  tgt.aux2_changed_on_dt = src.aux2_changed_on_dt,
  tgt.aux3_changed_on_dt = src.aux3_changed_on_dt,
  tgt.aux4_changed_on_dt = src.aux4_changed_on_dt,
  tgt.delete_flg = src.delete_flg,
  tgt.tenant_id = src.tenant_id,
  tgt.x_custom = src.x_custom,
  tgt.w_update_dt = current_timestamp(),
  tgt.etl_proc_wid = (SELECT ETL_PROC_WID FROM etl_params)
WHEN NOT MATCHED AND src.ind_update = 'I' THEN INSERT (
  ledger_name, ledger_short_name, ledger_desc, chart_of_accounts, currency_code,
  calender_name, sla_accounting_method_code, period_type, ledger_category_code, set_id,
  created_by_wid, changed_by_wid, created_on_dt, changed_on_dt,
  aux1_changed_on_dt, aux2_changed_on_dt, aux3_changed_on_dt, aux4_changed_on_dt,
  delete_flg, datasource_num_id, integration_id, tenant_id, x_custom,
  w_insert_dt, w_update_dt, etl_proc_wid
) VALUES (
  src.ledger_name, src.ledger_short_name, src.ledger_desc, src.chart_of_accounts, src.currency_code,
  src.calender_name, src.sla_accounting_method_code, src.period_type, src.ledger_category_code, src.set_id,
  src.created_by_wid, src.changed_by_wid, src.created_on_dt, src.changed_on_dt,
  src.aux1_changed_on_dt, src.aux2_changed_on_dt, src.aux3_changed_on_dt, src.aux4_changed_on_dt,
  src.delete_flg, src.datasource_num_id, src.integration_id, src.tenant_id, src.x_custom,
  current_timestamp(), current_timestamp(), (SELECT ETL_PROC_WID FROM etl_params)
);

In [ ]:
%sql
-- 5) UPDATE CONTROL TABLE
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SILOS_SIL_LEDGERDIMENSION' AS package_name,
    'W_LEDGER_D' AS target_table_name, p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID AS load_plan_id,
    current_timestamp() AS etl_load_date, CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date FROM etl_params p
) src ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.target_table_name = src.target_table_name, tgt.etl_load_date = src.etl_load_date, tgt.committed = src.committed
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name, src.etl_usage_code,
  src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
-- 6) INSERT HISTORY LOG
INSERT INTO workspace.oracle_edw.w_etl_load_dates_log
SELECT p.DATASOURCE_NUM_ID, 'SILOS_SIL_LEDGERDIMENSION', 'W_LEDGER_D', p.ETL_USAGE_CODE,
  p.ETL_PROC_WID, p.EXECUTION_ID, p.ETL_PROC_WID, date_add(current_timestamp(), -p.PRUNE_DAYS), NULL,
  current_timestamp(), CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM etl_params p;

In [ ]:
%sql
-- 7) CLEANUP
DROP VIEW IF EXISTS stg_with_change_ind;
DROP VIEW IF EXISTS stg_ledger_with_users;
DROP VIEW IF EXISTS etl_params;